<a href="https://colab.research.google.com/github/laffertybrian/food-sales-predictions/blob/main/food_sales_prediction_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####Food Sales Prediction Project

In [1]:
import pandas as pd

filename = '/content/drive/MyDrive/01 Python for Data Science/sales_predictions.csv'

sales_predictions_df = pd.read_csv(filename)
sales_predictions_df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


###Exploring the dataframe

In [2]:
# Finding the shape of the dataframe
sales_predictions_df.shape
# has 8523 rows and 12 columns

(8523, 12)

In [3]:
# Learning more about the datatypes of the variables
sales_predictions_df.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

In [4]:
# Checking data for duplicate values
sales_predictions_df.duplicated().sum()
# There are no duplicated rows

0

In [5]:
# Looking at the info of the dataframe
sales_predictions_df.info()
# Item weight and outlet size seem to have missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [6]:
# Another way to view missing values
sales_predictions_df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

###Addressing Missing Values

Understanding Outlet_Size and if other data within the dataframe could be used to fill the missing data.



In [7]:
# Viewing information contained within the Outlet_Size column
sales_predictions_df['Outlet_Size'].value_counts()

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

In [8]:
# Exploring data to see if data has a clue to correcting missing values
sales_predictions_df['Outlet_Type'].value_counts()

Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: Outlet_Type, dtype: int64

In [9]:
# Exploring outlet size to see if it can show us more about outlet type
sales_predictions_df.groupby(['Outlet_Size', 'Outlet_Type'])['Outlet_Size'].count()

Outlet_Size  Outlet_Type      
High         Supermarket Type1     932
Medium       Supermarket Type1     930
             Supermarket Type2     928
             Supermarket Type3     935
Small        Grocery Store         528
             Supermarket Type1    1860
Name: Outlet_Size, dtype: int64

In [10]:
# Exploring outlet size compared to outlet location type
sales_predictions_df.groupby(['Outlet_Size', 'Outlet_Location_Type'])['Outlet_Size'].count()

Outlet_Size  Outlet_Location_Type
High         Tier 3                   932
Medium       Tier 1                   930
             Tier 3                  1863
Small        Tier 1                  1458
             Tier 2                   930
Name: Outlet_Size, dtype: int64

In [11]:
# Putting the information known in the dataframe to help fill in missing values
sales_predictions_df.groupby(['Outlet_Size', 'Outlet_Type', 'Outlet_Location_Type'])['Outlet_Size'].count()

Outlet_Size  Outlet_Type        Outlet_Location_Type
High         Supermarket Type1  Tier 3                  932
Medium       Supermarket Type1  Tier 1                  930
             Supermarket Type2  Tier 3                  928
             Supermarket Type3  Tier 3                  935
Small        Grocery Store      Tier 1                  528
             Supermarket Type1  Tier 1                  930
                                Tier 2                  930
Name: Outlet_Size, dtype: int64

In [ ]:
#sales_predictions_df.groupby(['Outlet_Size', 'Outlet_Identifier', 'Outlet_Type', 'Outlet_Location_Type'])['Outlet_Size'].count()

After exploring the data it seems that columns Outlet_Type and Outlet_Location_Type can be used to potentially fill in the missing data from Outlet_Size.

In [12]:
# Getting a subset of data from the dataframe where supermarket type 1 and tier 3
# if there are any na values in the outlet size in this set they should be high
outlet_type_supermarket1 = sales_predictions_df['Outlet_Type'] == 'Supermarket Type1'
outlet_location_type_t3 = sales_predictions_df['Outlet_Location_Type'] == 'Tier 3'
test_high = sales_predictions_df.loc[outlet_type_supermarket1 & outlet_location_type_t3, :]

# none meet this criteria

outlet_type_supermarket2 = sales_predictions_df['Outlet_Type'] == 'Supermarket Type2'
test_medium2 = sales_predictions_df.loc[outlet_type_supermarket2, :]

# na outlet field search
outlet_size_na_filter = sales_predictions_df['Outlet_Size'].isna()
outlet_size_na = sales_predictions_df.loc[outlet_size_na_filter, :]



In [ ]:
# finding supermarket type1 and tier 2
outlet_location_type_t2 = sales_predictions_df['Outlet_Location_Type'] == 'Tier 2'
test_small_smt1_t2 = sales_predictions_df.loc[outlet_type_supermarket1 & outlet_location_type_t2, :]
test_small_smt1_t2.info()

In the dataframe Outlet_Size with the 'Small' value has certain creiteria based on Outlet_Type and Outlet_Location_Type. If there are missing values that also share these same criteria, it would be reasonable to fill in the missing values based on these findings.

No other Outlet_Size besides 'Small' contains the Outlet_Type equal to 'Grocery Store'. Also no other Outlet_Size besides 'Small' is associated with Outlet_Type 'Supermarket Type1' and Outlet_Location_Type 'Tier 2'.

In [ ]:
# testing to replace nan values with small in this category
test_small_smt1_t2['Outlet_Size'].fillna('Small', inplace = True)
test_small_smt1_t2.info()

In [15]:
# replaced nan values based on insight from the dataframe
# specifically 'Outlet_Type' to be Supermarket Type 1 and 'Outlet_Location_Size' to be Tier 2
df = sales_predictions_df
sales_predictions_df.loc[(df['Outlet_Type'] == 'Supermarket Type1') & (df['Outlet_Location_Type'] == 'Tier 2') & (df['Outlet_Size'].isnull()), 'Outlet_Size'] = 'Small'
sales_predictions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                7968 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [16]:
# searching for gorcery stores
grocery_filter = sales_predictions_df["Outlet_Type"] == "Grocery Store"
grocery_df = sales_predictions_df.loc[grocery_filter, :]
# Filling in the values of Outlet_Size based on Outlet_Type being equal to 'Grocery Store'
sales_predictions_df.loc[(df['Outlet_Type'] == 'Grocery Store') & (df['Outlet_Size'].isnull()), 'Outlet_Size'] = 'Small'
sales_predictions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                8523 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


After applying the patterns found in the data to the missing values in the Outlet_Size column, there are now 0 non-null values.

-- Now beginning to look at the missing data in the Item_Weight column.



In [17]:
# Getting information about the Item_Weight column
sales_predictions_df["Item_Weight"].describe()

count    7060.000000
mean       12.857645
std         4.643456
min         4.555000
25%         8.773750
50%        12.600000
75%        16.850000
max        21.350000
Name: Item_Weight, dtype: float64

In [18]:
# What percentage of the data from the Item Weight column is missing
1 - (7060 / 8523)

0.17165317376510614

I am choosing to replace the missing data in the Item_Weight column with the mean based on the flowing:

*   17% is relatively low 
*   I have never made a choice in a Grocery Store/Supermarket based on the items weight. (I am considering myslef a subject matter expert)



In [19]:
# Calculating the mean of the Item_Weight column
item_weight_mean = sales_predictions_df['Item_Weight'].mean()
# Applying the mean to the missing values in the column
sales_predictions_df['Item_Weight'].fillna(item_weight_mean, inplace = True)
# Checking that all values in the dataframe are non-null
sales_predictions_df.isna().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

All values are non-null in the dataframe.

###Addressing inconsistent category data

In [23]:
# Reviewing the data and looking at each column

# Found an issue with Item Fat Content
sales_predictions_df['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [34]:
# Correcting the syntax differences by using replace method

sales_predictions_df['Item_Fat_Content'] = sales_predictions_df['Item_Fat_Content'].replace('LF', 'Low Fat')
sales_predictions_df['Item_Fat_Content'] = sales_predictions_df['Item_Fat_Content'].replace('low fat', 'Low Fat')
sales_predictions_df['Item_Fat_Content'] = sales_predictions_df['Item_Fat_Content'].replace('reg', 'Regular')
# Printing the value counts again to check data within the column
sales_predictions_df['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

No other columns had incosistent data.

###Column Summary

Information about each column

In [45]:
# Information for Item Identifier
sales_predictions_df['Item_Identifier'].describe()

count      8523
unique     1559
top       FDW13
freq         10
Name: Item_Identifier, dtype: object

In [46]:
# Information about Item Weight
sales_predictions_df['Item_Weight'].describe()

count    8523.000000
mean       12.857645
std         4.226124
min         4.555000
25%         9.310000
50%        12.857645
75%        16.000000
max        21.350000
Name: Item_Weight, dtype: float64

In [50]:
# Information for Fat Content
sales_predictions_df['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [51]:
# Information for Item Visibility
sales_predictions_df['Item_Visibility'].describe()

count    8523.000000
mean        0.066132
std         0.051598
min         0.000000
25%         0.026989
50%         0.053931
75%         0.094585
max         0.328391
Name: Item_Visibility, dtype: float64

In [53]:
# Information for Item Type
# Counts of the different values
print(sales_predictions_df['Item_Type'].value_counts())
# Number of unique items, top item
sales_predictions_df['Item_Type'].describe()

Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64


count                      8523
unique                       16
top       Fruits and Vegetables
freq                       1232
Name: Item_Type, dtype: object

In [54]:
# Information about Item MRP
sales_predictions_df['Item_MRP'].describe()

count    8523.000000
mean      140.992782
std        62.275067
min        31.290000
25%        93.826500
50%       143.012800
75%       185.643700
max       266.888400
Name: Item_MRP, dtype: float64

In [36]:
# Information about Outlet Identifier
sales_predictions_df['Outlet_Identifier'].value_counts()

OUT027    935
OUT013    932
OUT049    930
OUT046    930
OUT035    930
OUT045    929
OUT018    928
OUT017    926
OUT010    555
OUT019    528
Name: Outlet_Identifier, dtype: int64

In [57]:
# Information about Outlet Establishment Year
sales_predictions_df['Outlet_Establishment_Year'].describe().round(2)

count    8523.00
mean     1997.83
std         8.37
min      1985.00
25%      1987.00
50%      1999.00
75%      2004.00
max      2009.00
Name: Outlet_Establishment_Year, dtype: float64

In [59]:
# Information about Outlet Size
sales_predictions_df['Outlet_Size'].value_counts()

Small     4798
Medium    2793
High       932
Name: Outlet_Size, dtype: int64

In [60]:
# Information about Outlet_Location_Type
sales_predictions_df['Outlet_Location_Type'].value_counts()

Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: Outlet_Location_Type, dtype: int64

In [43]:
# Information about Outlet Type
sales_predictions_df['Outlet_Type'].value_counts()

Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: Outlet_Type, dtype: int64

In [63]:
# Information about Item Outlet Sales
sales_predictions_df['Item_Outlet_Sales'].describe().round(2)

count     8523.00
mean      2181.29
std       1706.50
min         33.29
25%        834.25
50%       1794.33
75%       3101.30
max      13086.96
Name: Item_Outlet_Sales, dtype: float64